In [6]:
from py_conv import batch_conv2d
import numpy as np
import torch.nn.functional as F
import torch
from time import time

In [7]:
a = np.random.randn(3,4)
b = np.random.randn(3,4)
np.sum(a * b)

-4.193186151681344

In [8]:
inputs = np.random.randn(32,3,28,28)
weights = np.random.randn(8,3,5,5)
stride = (1,1)

In [9]:
start = time()
for i in range(1874):
    out_x = inputs.shape[2] - weights.shape[2] + 1
    out_y = inputs.shape[3] - weights.shape[3] + 1
    view_shape = (1, out_x, out_y, weights.shape[1], weights.shape[2], weights.shape[3])
    _strides = inputs[0].data.strides*2
    strided_batch = np.array([np.lib.stride_tricks.as_strided(x, view_shape, _strides) for x in inputs])[:, 0, 0:out_x:stride[0], 0:out_y:stride[1]]
    result1 = batch_conv2d(strided_batch,weights, [32,8,24,24]).reshape([32,8,24,24])
c_used = time() - start

torch_inputs = torch.tensor(inputs)
torch_weights = torch.tensor(weights)
torch_bias = torch.zeros(8)
start = time()
for i in range(1874):
    result2 = F.conv2d(torch_inputs, torch_weights, torch_bias)
torch_used = time() - start

In [10]:
print(f"C Function Used -> {c_used}")
print(f"Pytorch Used    -> {torch_used}")

C Function Used -> 34.220893144607544
Pytorch Used    -> 5.736648797988892


In [6]:
# Assert Result
np.testing.assert_almost_equal(result1, result2)

C Function Used -> 6.641327142715454  
Pytorch Used    -> 1.343163251876831
